# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


In [2]:
import pandas as pd
#импортирование библиотеки

In [3]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


пропущены значения типа float64 в столбцах days_employed & total_income. 

### Шаг 2.1 Заполнение пропусков

In [4]:
display(data.isna().sum())
display(data.head(10))

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
#проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
conversion_days_employed = data['days_employed'].isna().sum()/data['days_employed'].count()
display('Доля пропущенных значений в столбце days_employed: {:.0%}'.format(conversion_days_employed))
conversion_total_income = data['total_income'].isna().sum()/data['total_income'].count()
display('Доля пропущенных значений в столбце total_income: {:.0%}'.format(conversion_total_income))

'Доля пропущенных значений в столбце days_employed: 11%'

'Доля пропущенных значений в столбце total_income: 11%'

Причинами пропусков (NaN) могут быть ошибки ввода данных, сокрытие информации. Вданном датасете в первых 10 строках в столбце days_employed можно увидеть отрицательные значения, что говорит о неверном вводе данных. В столбце total_income возможна такая же ошибка, но так же люди неохотно делятся суммой своих доходов, так что в данном случае пропуски NaN могут быть обусловлены сокрытием информации в дополнении к ошибке ввода данных.

In [6]:
#найдем ср. знач и медиану дохода и стажа соотв.
days_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_avg)
total_income_avg = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value = total_income_avg)
#data['days_employed'] = data['days_employed'].fillna(data.groupby(['days_employed'])['days_employed'].transform('median'))
#data['total_income'] = data['total_income'].fillna(data.groupby(['income_type'])['total_income'].transform('median'))
display(data.isna().sum())


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Медианное значение в отлиции от Среднего более точно характеризует пропуски в количественных переменных.

Шаг 2.2. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных.

In [7]:
data['days_employed'] = abs(data['days_employed'])
data['children'] = abs(data['children'])

data['children'] = data['children'].replace(20, 2) #заменим 20 детей на 2, т.к. это с большой вероятностью опечатка, выброс. 
data['children'].describe()

count    21525.000000
mean         0.479721
std          0.755528
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

### Шаг 2.3. Изменение типов данных.

In [8]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data.info()
#display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

In [9]:
# найдем кол-во строк-дубликатов 
data.duplicated().sum()
#duplicated_data = data[data.duplicated()]
#display(duplicated_data)

54

In [10]:
#удаленние строк дубликатов
data = data.drop_duplicates().reset_index(drop = True) #удаление строк дубликатовб создание новой индексации и удаление старой
data.duplicated().sum()
#duplicated_data_new = data[data.duplicated()] #проверка/поиск дубликотов после удаления
#display(duplicated_data_new)

0

In [11]:
# поиск неявных дубликотов в столбце education
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [12]:
data['education'] = data['education'].str.lower() 
data['education'].unique()
#display(data.head(30)) 

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [13]:
new_df_education = data[['education', 'education_id']]
new_df_family_status = data[['family_status', 'family_status_id']]

new_df_education =  data[['education', 'education_id']].drop_duplicates().reset_index(drop = True) 
#удаление строк дубликатовб создание новой индексации и удаление старой
new_df_family_status =  data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop = True) 
data.duplicated().sum()

display(new_df_education.head())
display(new_df_family_status.head())

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [14]:
#sorted(data)
#data = data[['children',
# 'days_employed',
# 'debt',
# 'dob_years',
# 'education_id',
# 'family_status_id',
# 'gender',
# 'income_type',
# 'purpose',
# 'total_income']]
data.drop(columns = ['education'], axis = 1, inplace=True)
data.drop(columns = ['family_status'], axis = 1, inplace=True)
display(data)
#data.drop('education', axis=1) еще один метод исключения столбца из нового датафрейма


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

In [15]:
#data['total_income_category'] 
def categorize_income(income_value):
    if income_value <= 30000:
        return 'E'
    elif income_value <= 50000:
        return 'D'
    elif income_value <= 200000:
        return 'C'
    elif income_value <= 1000000:
        return 'B'
    else:
        return 'A'

data['total_income_category'] = data['total_income'].apply(categorize_income)
data['total_income_category'].value_counts()


C    16033
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [16]:
def categorize_purpose(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'образован' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью' 
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data['purpose_category'].value_counts()


операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [17]:
df_children = pd.DataFrame()
df_children['sum_children'] = data.groupby('children')['debt'].sum()
df_children['count_children'] = data.groupby('children')['debt'].count()
df_children['results_children'] = df_children['sum_children']/df_children['count_children']*100
df_children.sort_values('results_children', ascending =False)
display(df_children)

correcaltion_children = pd.DataFrame()
correcaltion_children['debt'] = data['debt']
correcaltion_children['children'] = data['children']

correcaltion_children.corr()

,sum_children,count_children,results_children
children,,,
0,1063,14107,7.535266
1,445,4856,9.163921
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


,debt,children
debt,1.000000,0.024812
children,0.024812,1.000000


Вывод:  Данная зависимость показывает, что клиенты без детей отвественнее всего подходят к выплате кредита в срок, впринципе, в этом есть логика, так как у них меньше ежемесячных затрат. На втором месте расположились клиенты с 3-я детьми, мои предположения в том, что семьи с 3-я детьми более отвественны, чем семьи с 1-2 детьми, так как им наверняка потребуется хорошая кредитная история для будущих кредитов, например для детей. На 3 ,4 и 5 местах расположились семьи с 1-м, 2-я и 4-я детьм соотвественно, опираясь на логику описанную в предидущем предложении, могу предположить, что семьи с 1-м и 2-я детьми берут кредит одноразово, поэтому не так сильно ориентируются на срок выплат. 

Вопрос 2- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
#df = data.groupby('family_status_id', as_index=False).debt.mean()
df_family = pd.DataFrame()
df_family['family_status_count'] = data.groupby('family_status_id')['debt'].count()
df_family['family_status_sum'] = data.groupby('family_status_id')['debt'].sum()
df_family['results_family_status'] = df_family['family_status_sum']/df_family['family_status_count']*100
df_family.sort_values('results_family_status', ascending = False)
display(df_family)


#family_status_id(0) = женат / замужем
#family_status_id(1) = гражданский брак
#family_status_id(2) = вдовец / вдова
#family_status_id(3) = в разводе
#family_status_id(4) = Не женат / не замужем

,family_status_count,family_status_sum,results_family_status
family_status_id,,,
0,12344,931,7.542126
1,4163,388,9.320202
2,959,63,6.569343
3,1195,85,7.112971
4,2810,274,9.750890


Вывод:  Данные показывают, что при семейном положении "Не женат / не замужем" вероятность невозврата кредита самая высокая,
тогда как при семейном положении "вдовец / вдова" вероятность невозврата кредита самая низкая. Разница между минимальным значением 6.56% и максимальным 9.75% составляет 32,6% процентов.

Вопрос 3- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
#data['total_income_category'] = data['total_income'].apply(total_income_category)
#data['total_income_category'].value_counts()
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = data.groupby('total_income_category')['debt'].sum()
debt_from_total_income['count'] = data.groupby('total_income_category')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']*100
debt_from_total_income.sort_values('conversion', ascending = False)
display(debt_from_total_income)
display(data['total_income_category'])
verification_through_correlation = pd.DataFrame()
verification_through_correlation['total_income'] = data['total_income']
verification_through_correlation['debt'] = data['debt']
verification_through_correlation.corr(


,sum,count,conversion
total_income_category,,,
A,2,25,8.000000
B,356,5041,7.062091
C,1360,16033,8.482505
D,21,350,6.000000
E,2,22,9.090909


0        B
1        C
2        C
3        B
4        C
        ..
21466    B
21467    C
21468    C
21469    B
21470    C
Name: total_income_category, Length: 21471, dtype: object

,total_income,debt
total_income,1.000000,-0.011748
debt,-0.011748,1.000000


Категоризовав уровень дохода на 5 категорий в результате видим, что зависимость между уровнем дохода и возвратом кредита в срок не наблюдается. Но несмотря на то, что имеется 5 категирий, две выборки: В(до 1000000) и С(до 200000) заметно превышают по кол-ву остальные, если судить только по этим самым большим выборкам, то при большем доходе (категория В) существует меньший процент по невозврату кредита в срок, чем при меньшем доходе (категирия С).

Вопрос 4 - Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_purpose = pd.DataFrame()
df_purpose['sum_income'] = data.groupby('purpose_category')['debt'].sum()
df_purpose['count_income'] = data.groupby('purpose_category')['debt'].count()
df_purpose['results_income'] = df_purpose['sum_income']/df_purpose['count_income'] 
df_purpose.sort_values('results_income', ascending =False)
display(df_purpose)
#data.groupby('purpose_category')['debt'].mean() 


Зависимость между разными целями кредита и возвратом его в срок следущая: процент ненадежных заемщиков в операциях с недвижимостью самый минимальный из включенных категорий, тогда как максимальный процент ненадежных заемщиков находится при операциях с автомобилем. 

## Общий вывод:

В ходе данного исследования мы выяснили:
<br>
1) что наличие детей увеличивает шансы на возврат кредита в срок. Идеальным клиентом считается: семья с 5-ю детьми. Худшим клиентом является:  семья с (4) - детьми.  
<br>
2) При семейном положении Не женат / не замужем наблюдается самый высокий процент по невозврату кредита в срок. Идеальным клиентом считается: Вдова/вдовец.
<br>
3) Сравнив разные категории по уровню З/П, мы видим, что зависимости между уровнем дохода и возвратом кредита в срок не наблюдается. Но несмотря на то, что имеется 5 категирий, две выборки: В(до 1000000) и С(до 200000) заметно превышают по кол-ву остальные, если судить только по этим (самым большим) выборкам, то при большем доходе (категория В) существует меньший процент по невозврату кредита в срок, чем при меньшем доходе (категирия С).
<br>
4) Максимальный процент ненадежных заемщиков находится при операциях с автомобилем. Идеальным клиентом считается: человек c целью покупки недвижимости.

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.
 